In [1]:
import import_ipynb
import logging
import numpy as np
from optparse import OptionParser
import os
import sys
from time import time
import matplotlib.pyplot as plt
import random  
from random import sample 
import pandas as pd
import pickle
import scipy

from mlxtend.classifier import StackingClassifier
from sklearn.decomposition import PCA, NMF, KernelPCA, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

from My_Transformers import *
from Word2Vec_Transformer import WVEmbeddingTransformer, Text_cleaner_transformer, EMB_VOCAB_SET, WV_EMBEDDINGS, getCosineNearestNeighbours

import warnings
warnings.filterwarnings("ignore")

importing Jupyter notebook from My_Transformers.ipynb
importing Jupyter notebook from Word2Vec_Transformer.ipynb


2020-09-12 23:59:56,885 INFO adding document #0 to Dictionary(0 unique tokens: [])
2020-09-12 23:59:56,886 INFO built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [2]:
def pickleSave (obj, file):
    
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

In [3]:
def unpickle (file):
    return pickle.load (open (file, 'rb'))

In [4]:
def preProcess (rawfile, newFileName):
    
    df = pd.read_csv (rawfile)
    Xstr = df['text']
    text_cleaner = Text_cleaner_transformer (isCorrection=True, embedding=WV_EMBEDDINGS)
    Xstr = text_cleaner.transform (Xstr)
    df['text'] = Xstr
    df.to_csv (newFileName)
    return

In [5]:
# init the WVEmbeddingTransformer only once
wvEmbeddingTransformer = WVEmbeddingTransformer ('D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin', vocabSet=EMB_VOCAB_SET, isCorrection=True)

2020-09-12 23:59:58,803 INFO loading projection weights from D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin
2020-09-13 00:00:47,627 INFO loaded (3000000, 300) matrix from D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin


In [6]:
train_file = "Data/train.csv"
preprocessed_train_file = "Data/preprocessed_train.csv"
preProcess (train_file, preprocessed_train_file)

In [7]:
trainDF = pd.read_csv (preprocessed_train_file)
# trainDF = trainDF[0:5]
trainDF.shape

(7613, 6)

In [8]:
X_train_str = trainDF["text"]
y_train = trainDF["target"]

In [9]:
X_train_emb = wvEmbeddingTransformer.transform (X_train_str)  # fit_transform (Xstr)
pickleSave (X_train_emb, 'X_train_emb.bin')
# X_train_emb = unpickle ('X_train_emb.bin')
X_train_str = None          # free memory

In [10]:
test_file = "Data/test.csv"
preprocessed_test_file = "Data/preprocessed_test.csv"
preProcess (test_file, preprocessed_test_file)

In [11]:
testDF = pd.read_csv (preprocessed_test_file)
# testDF = testDF[0:5]
X_test_str = testDF["text"]
testDF.shape

(3263, 5)

In [12]:
X_test_emb = wvEmbeddingTransformer.transform (X_test_str)
pickleSave (X_test_emb, 'X_test_emb.bin')
# X_test_emb = unpickle ('X_test_emb.bin')
X_test_str = None    # free memory

In [20]:
targets = []
indexes = []
for q_vec in X_test_emb:
    index = getCosineNearestNeighbours (q_vec, X_train_emb, 1, wvEmbeddingTransformer.dim)[0]
    indexes.append (index)
    targets.append (trainDF['target'][index])

In [21]:
testDF['target'] = targets
submitDF = testDF[['id', 'target']]
submitDF.to_csv ('submission_v2.csv', index=False)

In [22]:
submitDF

,id,target
0,0,1
1,2,1
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1
